# Unblock Me (CLI) — Experimentos comparativos

Este notebook ejecuta A*, BFS, DFS y UCS sobre el mismo tablero y compara:
- Movimientos (longitud del plan)
- Nodos expandidos
- Tiempo de resolución

Ejecuta las celdas en orden. El notebook incluye el código del juego (sucesores, heurística, algoritmos) y una sección final donde se grafican las comparativas con matplotlib.

In [ ]:
!pip install pandas

In [ ]:
# %% Cell: imports y utilidades
import time
from heapq import heappush, heappop
from collections import deque
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

ROWS, COLS = 6, 6

class Block:
    def __init__(self, x, y, length, orientation, is_red=False):
        self.x = x
        self.y = y
        self.length = length
        self.orientation = orientation  # 'H' o 'V'
        self.is_red = is_red

    def copy(self):
        return Block(self.x, self.y, self.length, self.orientation, self.is_red)

def blocks_to_state(blocks):
    return tuple((b.x, b.y) for b in blocks)

def apply_state_to_blocks(state, blocks):
    for (x, y), b in zip(state, blocks):
        b.x, b.y = x, y


In [ ]:
# %% Cell: nivel inicial y meta data
START_BLOCKS = [
    Block(1, 2, 2, 'H', True),   # rojo (índice 0)
    Block(3, 0, 3, 'V'),
    Block(0, 0, 2, 'V'),
    Block(5, 0, 3, 'V'),
    Block(0, 4, 3, 'H'),
    Block(2, 5, 3, 'H'),
]

LEVEL_META = []
RED_IDX = 0

def init_level_meta_from(blocks):
    global LEVEL_META, RED_IDX
    LEVEL_META = [(b.length, b.orientation, b.is_red) for b in blocks]
    RED_IDX = 0
    for i, meta in enumerate(LEVEL_META):
        if meta[2]:
            RED_IDX = i
            break

# Inicializar meta global con START_BLOCKS
init_level_meta_from(START_BLOCKS)


In [5]:
# %% Cell: grid, meta y heurística
def build_grid(state):
    grid = [[-1 for _ in range(COLS)] for _ in range(ROWS)]
    for idx, (x, y) in enumerate(state):
        length, orient, _ = LEVEL_META[idx]
        if orient == 'H':
            for i in range(length):
                grid[y][x+i] = idx
        else:
            for i in range(length):
                grid[y+i][x] = idx
    return grid

def is_goal(state):
    rx, ry = state[RED_IDX]
    rlen, _, _ = LEVEL_META[RED_IDX]
    red_right = rx + rlen - 1
    return red_right == COLS - 1

def move_cost(_from, _to):
    return 1

def heuristic(state):
    if is_goal(state):
        return 0
    rx, ry = state[RED_IDX]
    rlen, _, _ = LEVEL_META[RED_IDX]
    red_right = rx + rlen - 1
    blockers = set()
    for cx in range(red_right + 1, COLS):
        for j, (bx, by) in enumerate(state):
            blen, borient, _ = LEVEL_META[j]
            if j == RED_IDX:
                continue
            if borient == 'H':
                if by == ry and bx <= cx <= bx + blen - 1:
                    blockers.add(j)
            else:
                if bx == cx and by <= ry <= by + blen - 1:
                    blockers.add(j)
    return len(blockers) + 1


In [6]:
# %% Cell: sucesores
def successors(state):
    succs = []
    grid = build_grid(state)
    for i, (x, y) in enumerate(state):
        length, orient, _ = LEVEL_META[i]
        if orient == 'H':
            step = 1
            while x - step >= 0 and grid[y][x - step] == -1:
                new = list(state); new[i] = (x - step, y)
                succs.append(tuple(new)); step += 1
            step = 1
            right_end = x + length - 1
            while right_end + step <= COLS - 1 and grid[y][right_end + step] == -1:
                new = list(state); new[i] = (x + step, y)
                succs.append(tuple(new)); step += 1
        else:
            step = 1
            while y - step >= 0 and grid[y - step][x] == -1:
                new = list(state); new[i] = (x, y - step)
                succs.append(tuple(new)); step += 1
            step = 1
            bottom_end = y + length - 1
            while bottom_end + step <= ROWS - 1 and grid[bottom_end + step][x] == -1:
                new = list(state); new[i] = (x, y + step)
                succs.append(tuple(new)); step += 1
    return succs


In [7]:
# %% Cell: reconstrucción
def reconstruct(parent_map, goal_state):
    path = [goal_state]
    cur = goal_state
    while parent_map[cur] is not None:
        cur = parent_map[cur]
        path.append(cur)
    path.reverse()
    return path


In [9]:
# %% Cell: algoritmos (A*, BFS, DFS, UCS)
def astar(start_state, max_exp=400000):
    class Node:
        __slots__ = ("state","g","f")
        def __init__(self, state, g):
            self.state = state
            self.g = g
            self.f = g + heuristic(state)
        def __lt__(self, other):
            if self.f != other.f: return self.f < other.f
            return self.g > other.g

    open_heap = []
    heappush(open_heap, Node(start_state, 0))
    best_g = {start_state: 0}
    parent = {start_state: None}
    expansions = 0

    while open_heap and expansions <= max_exp:
        current = heappop(open_heap)
        s = current.state
        if is_goal(s):
            return reconstruct(parent, s), expansions
        if current.g > best_g.get(s, float('inf')):
            continue
        expansions += 1
        for nxt in successors(s):
            new_g = current.g + move_cost(s, nxt)
            if new_g < best_g.get(nxt, float('inf')):
                best_g[nxt] = new_g
                parent[nxt] = s
                heappush(open_heap, Node(nxt, new_g))
    return None, expansions

def bfs(start_state, max_exp=600000):
    if is_goal(start_state):
        return [start_state], 0
    Q = deque([start_state])
    parent = {start_state: None}
    expansions = 0
    while Q and expansions <= max_exp:
        s = Q.popleft()
        expansions += 1
        for nxt in successors(s):
            if nxt not in parent:        # parent como visited
                parent[nxt] = s
                if is_goal(nxt):
                    return reconstruct(parent, nxt), expansions
                Q.append(nxt)
    return None, expansions

def dfs(start_state, max_exp=600000):
    stack = [start_state]
    parent = {start_state: None}
    visited = {start_state}
    expansions = 0
    while stack and expansions <= max_exp:
        s = stack.pop()
        if is_goal(s):
            return reconstruct(parent, s), expansions
        expansions += 1
        for nxt in successors(s):
            if nxt not in visited:
                visited.add(nxt)
                parent[nxt] = s
                stack.append(nxt)
    return None, expansions

def ucs(start_state, max_exp=600000):
    heap = []
    heappush(heap, (0, start_state))
    best_g = {start_state: 0}
    parent = {start_state: None}
    expansions = 0
    while heap and expansions <= max_exp:
        g, s = heappop(heap)
        if g > best_g.get(s, float('inf')):
            continue
        if is_goal(s):
            return reconstruct(parent, s), expansions
        expansions += 1
        for nxt in successors(s):
            new_g = g + move_cost(s, nxt)
            if new_g < best_g.get(nxt, float('inf')):
                best_g[nxt] = new_g
                parent[nxt] = s
                heappush(heap, (new_g, nxt))
    return None, expansions


In [ ]:
# %% Cell: runner que mide métricas
def run_algorithm(alg_fn, start_state):
    t0 = time.perf_counter()
    sol, expansions = alg_fn(start_state)
    t1 = time.perf_counter()
    elapsed = t1 - t0
    moves = None
    if sol is None:
        moves = None
    else:
        moves = max(0, len(sol) - 1)
    return {"moves": moves, "expansions": expansions, "time": elapsed, "solution": sol}

def run_all(start_blocks=None, repeat=1, algorithms=None):
    if start_blocks is None:
        start_blocks = START_BLOCKS
    if algorithms is None:
        algorithms = [("A*", astar), ("BFS", bfs), ("DFS", dfs), ("UCS", ucs)]
    init_level_meta_from(start_blocks)
    start_state = blocks_to_state([b.copy() for b in start_blocks])
    results = []
    for name, fn in algorithms:
        measures = {"moves": [], "expansions": [], "time": []}
        sol_example = None
        for _ in range(repeat):
            r = run_algorithm(fn, start_state)
            measures["moves"].append(r["moves"])
            measures["expansions"].append(r["expansions"])
            measures["time"].append(r["time"])
            if sol_example is None and r["solution"] is not None:
                sol_example = r["solution"]
        moves_vals = [m for m in measures["moves"] if m is not None]
        avg_moves = np.mean(moves_vals) if len(moves_vals)>0 else None
        results.append({
            "algorithm": name,
            "moves_mean": avg_moves,
            "moves_raw": measures["moves"],
            "expansions_mean": float(np.mean(measures["expansions"])),
            "expansions_raw": measures["expansions"],
            "time_mean": float(np.mean(measures["time"])),
            "time_raw": measures["time"],
            "solution_example": sol_example
        })
    df = pd.DataFrame(results)
    return df

# Ejemplo rápido (repeat=1)
df = run_all(repeat=1)
df

In [ ]:
# %% Cell: imprimir tabla de resultados
pd.set_option("display.precision", 6)
df_display = df.copy()
df_display["moves_mean"] = df_display["moves_mean"].apply(lambda x: int(x) if pd.notna(x) else "Sin solución")
df_display[["algorithm","moves_mean","expansions_mean","time_mean"]]


In [ ]:
# %% Cell: graficar comparativas con matplotlib
algorithms = df["algorithm"].tolist()
moves = [np.nan if pd.isna(v) else v for v in df["moves_mean"].tolist()]
expans = df["expansions_mean"].tolist()
times = df["time_mean"].tolist()

x = np.arange(len(algorithms))
width = 0.6

fig, axs = plt.subplots(1, 3, figsize=(16,5))

axs[0].bar(x, moves, width)
axs[0].set_xticks(x); axs[0].set_xticklabels(algorithms)
axs[0].set_title("Movimientos (plan length)")
axs[0].set_ylabel("transiciones")

axs[1].bar(x, expans, width)
axs[1].set_xticks(x); axs[1].set_xticklabels(algorithms)
axs[1].set_title("Nodos expandidos")
axs[1].set_ylabel("nodos")

axs[2].bar(x, times, width)
axs[2].set_xticks(x); axs[2].set_xticklabels(algorithms)
axs[2].set_title("Tiempo de resolución")
axs[2].set_ylabel("segundos")

plt.tight_layout()
plt.show()
